In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
from keras.models import load_model
import json
import math
import numpy as np
from sklearn.metrics import confusion_matrix
def dh1(x):
	temp=np.zeros(x.shape)
	temp[:]=x[:]
	for i in range(len(x[:,-1])-2):
		x[i+1,-1]=int((temp[i+2,-1]-temp[i,-1])/2)
	return np.expand_dims(x[1:len(x)-1,-1], 1)
def standardd(X):
	Xnew=np.asarray(X,dtype='float')
	total=0
	u=X.sum()/len(X)
	for i in range(len(X)):
		total+=((X[i]-u)**2)
	o=math.sqrt(total/len(X))
	for i in range(len(X)):
		Xnew[i]=float((X[i]-u)/o)
	return Xnew
def processdata1(X_data):
  X_datanew=[]
  for i in range(X_data.shape[0]):
	  X=dh1(X_data[i])
	  X_datanew.append(X)
  X_datanew=np.asarray(X_datanew)
  for i in range(X_datanew.shape[0]):
	  X_datanew[i]=standardd(X_datanew[i])
  return X_datanew
def processdata2(X_data):
  for i in range(X_data.shape[0]):
    X_data[i]=standardd(X_data[i])
  return X_data

testing = np.load('drive/My Drive/ML_tutorial/testinghigh.npz')
X_data = testing['xdata']
y_true = testing['ytrue']
#print(y_true)
X_test1=processdata1(X_data)
X_test2=processdata2(X_data)

mymodel = load_model("drive/My Drive/ML_tutorial/Weight/modelhigh2526.h5")

with open('./drive/My Drive/ML_tutorial/label_dict.json', 'r') as label_file:
    label_dict = json.loads(label_file.read())
print(label_dict)

y_pred=mymodel.predict([X_test1,X_test2])

y_pred_o=[]
for i in range(len(y_pred)):
  maxx=np.argmax(y_pred[i])
  y_pred_o.append(label_dict[maxx])
y_pred_o=np.asarray(y_pred_o)
cnf_matrix = confusion_matrix(y_true, y_pred_o)
print(cnf_matrix)
correct=0
for i in range(len(y_true)):
  if y_pred_o[i]==y_true[i]:
    correct+=1
  else:
    print('{} but predicted {}'.format(y_true[i], y_pred_o[i]))
print('test_acc: {0:.1f}%'.format(correct * 100/len(y_true)))

['Apple', 'Avocado', 'Banana', 'DragonFruit', 'Guava', 'Lychee', 'Mango', 'Potato', 'Tomato']
[[65  0  0  0  0  1]
 [ 0 64  0  2  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  1 64  1  0]
 [ 0  0  0  0 66  0]
 [ 0  0  0  0  0 66]]
Apple but predicted Mango
Avocado but predicted DragonFruit
Avocado but predicted DragonFruit
DragonFruit but predicted Guava
DragonFruit but predicted Banana
test_acc: 98.5%
